# **STT API Tutorial**

This notebook provides a step-by-step guide on how to use the STT API for speech-to-text tasks. It includes instructions for installation, setting up the API key, uploading audio files, and using the API for transcription and translation.*italicised text*



### **1. Installation**
Before you begin, ensure you have the necessary Python libraries installed. Run the following commands to install the required packages:


In [25]:
!pip install requests pandas pydub


### **1. Import Required Libraries**

This section imports the necessary Python libraries for making HTTP requests, handling audio files, and managing data.

In [26]:
import requests
import pandas as pd
from pydub import AudioSegment
import io

### **2. Set Up the API Endpoint and Payload**

To use the Saaras API, you need an API subscription key. Follow these steps to set up your API key:

1. **Obtain your API key**: If you don’t have an API key, sign up on the [Sarvam AI Dashboard](https://dashboard.sarvam.ai/) to get one.
2. **Replace the placeholder key**: In the code below, replace "YOUR_SARVAM_AI_API_KEY" with your actual API key.

In [27]:
SARVAM_AI_API="YOUR_SARVAM_AI_API_KEY"

### ***2.1 Setting Up the API Endpoint and Payload***

This section defines the API endpoint and the payload for the translation request. Replace the placeholder values with your actual API key and desired parameters.

In [28]:
# API endpoint for speech-to-text
api_url = "https://api.sarvam.ai/speech-to-text"

# Headers containing the API subscription key
headers = {
    "api-subscription-key": SARVAM_AI_API  # Replace with your API key
}

# Data payload for the transcription request
data = {
    "language_code": "hi-IN",  # Specify the language of the audio (e.g., 'hi-IN' for Hindi)
    "model": "saarika:v2",     # Specify the model to be used for transcription
    "with_timestamps": False   # Set to True if you want word-level timestamps
}

## **3. Uploading Audio Files**

To translate audio, you need to upload a `.wav` file. Follow these steps:

1. **Prepare your audio file**: Ensure your audio file is in `.wav` format. If your file is in a different format, you can use tools like `pydub` to convert it.
2. **Upload the file**: If you're using Google Colab, you can upload the file using the file uploader:

In [29]:
from google.colab import files

uploaded = files.upload()
audio_file_path = list(uploaded.keys())[0]  # Get the name of the uploaded file

Saving test.wav to test (2).wav


### **4. Define the `split_audio` Function**

This function splits an audio file into smaller chunks of a specified duration. This is useful for processing long audio files that exceed the API's input length limit.

In [30]:
def split_audio(audio_path, chunk_duration_ms):
    """
    Splits an audio file into smaller chunks of specified duration.

    Args:
        audio_path (str): Path to the audio file to be split.
        chunk_duration_ms (int): Duration of each chunk in milliseconds.

    Returns:
        list: A list of AudioSegment objects representing the audio chunks.
    """
    audio = AudioSegment.from_file(audio_path)  # Load the audio file
    chunks = []
    if len(audio) > chunk_duration_ms:
        # Split the audio into chunks of the specified duration
        for i in range(0, len(audio), chunk_duration_ms):
            chunks.append(audio[i:i + chunk_duration_ms])
    else:
        # If the audio is shorter than the chunk duration, use the entire audio
        chunks.append(audio)
    return chunks

### **5. Define the `transcribe_audio_chunks` Function**

This function transcribes audio chunks using the Saaras API. It handles the API request for each chunk and collates the results.
**bold text**

In [31]:
def transcribe_audio_chunks(audio_file_path, api_url, headers, data, chunk_duration_ms=5*60*1000):
    """
    Transcribes audio chunks using the Speech-to-Text API.

    Args:
        audio_file_path (str): Path to the audio file.
        api_url (str): The API endpoint URL for Speech-to-Text.
        headers (dict): Headers containing authentication information.
        data (dict): Data payload for the transcription API.
        chunk_duration_ms (int): Duration of each audio chunk in milliseconds.

    Returns:
        dict: Collated response containing the transcript.
    """
    # Split the audio into chunks
    chunks = split_audio(audio_file_path, chunk_duration_ms)
    responses = []  # List to store the transcription results

    # Process each chunk
    for idx, chunk in enumerate(chunks):
        # Export the chunk to a BytesIO object (in-memory binary stream)
        chunk_buffer = io.BytesIO()
        chunk.export(chunk_buffer, format="wav")
        chunk_buffer.seek(0)  # Reset the pointer to the start of the stream

        # Prepare the file for the API request
        files = {'file': ('audiofile.wav', chunk_buffer, 'audio/wav')}

        try:
            # Make the POST request to the API
            response = requests.post(api_url, headers=headers, files=files, data=data)
            if response.status_code == 200 or response.status_code == 201:
                print(f"Chunk {idx} POST Request Successful!")
                response_data = response.json()
                transcript = response_data.get("transcript", "")
                responses.append({"transcript": transcript})
            else:
                # Handle failed requests
                print(f"Chunk {idx} POST Request failed with status code: {response.status_code}")
                print("Response:", response.text)
        except Exception as e:
            # Handle any exceptions during the request
            print(f"Error processing chunk {idx}: {e}")
        finally:
            # Ensure the buffer is closed after processing
            chunk_buffer.close()

    # Collate the transcriptions from all chunks
    collated_responses = {"collated_transcript": " ".join([i["transcript"] for i in responses])}
    return collated_responses

### **6. Transcribe the Audio**

This section calls the `transcribe_audio_chunks` function to transcribe the audio file. Replace `audio_file_path` with the path to your audio file

In [32]:
# Path to the audio file to be transcribed
# audio_file_path = "test.wav"  # Replace with your file path

# Transcribe the audio
transcriptions = transcribe_audio_chunks(audio_file_path, api_url, headers, data)

# Display the transcription results
transcriptions

Chunk 0 POST Request Successful!


{'collated_transcript': 'अंग्रेज़ी में बच्चों के लिए लाभ से प्रपूर्ण अनेक नैतिक कहानियाँ हैं। वे आपके बच्चे की कल्पना को सक्रिय करते हैं, उनका मनोरंजन करते हैं और उन्हें खुश करते हैं। फूटी नैतिक कहानियाँ उनका ध्यान बनाए रखने और पूरी कहानी के दौरान उन्हें ध्यान केंद्रित रखने के लिए आदर्श हैं।'}

### **6. Explanation of the Output**

The output of the `transcribe_audio_chunks` function is a dictionary containing the collated transcript of the entire audio file. If the audio was split into multiple chunks, the transcripts from all chunks are combined into a single string.

Example output:

```{
    "collated_transcript": "This is the transcribed text from the audio file."
}```

### **7. Conclusion**

This tutorial demonstrated how to use the **STT API** for speech-to-text transcription. By following the steps, you can transcribe audio files, even long ones, by splitting them into smaller chunks. The process involves installing required libraries, setting up your API key, uploading audio, and transcribing it using the provided functions.

---

### **8. Additional Resources**

For more details, refer to the our official documentation and we are always there to support and help you on our Discord Server:

- **Documentation**: [docs.sarvam.ai](https://docs.sarvam.ai)  
- **Community**: [Join the Discord Community](https://discord.gg/hTuVuPNF)

---

### **9. Final Notes**

- Keep your API key secure.
- Use clear audio for best results.
- Explore advanced features like diarization and translation.

**Keep Building!** 🚀

## Using Saaras for subtitles

you can easily overcome the one-inch tall barrier of subtitles using Saaras. Generate custom subtitles for your audios using a combination of Voice Activity Detection and Saaras's output and save the subtitles in an .srt file for further use.

In [ ]:
import os
import torch
import numpy as np
import librosa
import soundfile as sf
from tqdm import tqdm

sample_rate = 16000  # Set the sample rate for loading audio
vad_threshold = 0.5  # Threshold for VAD
combine_duration = 8  # Maximum duration for combined segments
combine_gap = 1  # Maximum gap between segments to combine

@torch.no_grad()
def get_vad_probs(model, audio, sample_rate=16000):
    audio = torch.as_tensor(audio, dtype=torch.float32)
    window_size_samples = 512 if sample_rate == 16000 else 256

    model.reset_states()
    audio_length_samples = len(audio)

    speech_probs = []
    for current_start_sample in range(0, audio_length_samples, window_size_samples):
        chunk = audio[current_start_sample: current_start_sample + window_size_samples]
        if len(chunk) < window_size_samples:
            chunk = torch.nn.functional.pad(chunk, (0, int(window_size_samples - len(chunk))))
        speech_prob = model(chunk, sample_rate).item()
        speech_probs.append(speech_prob)

    return speech_probs

def get_utterances(vad_probs, threshold=0.5, frame_duration=0.032):
    """Extracts utterances (start and end times) based on VAD probabilities."""
    utterances = []
    in_utterance = False
    utterance_start = 0

    for i, prob in enumerate(vad_probs):
        if prob > threshold and not in_utterance:
            in_utterance = True
            utterance_start = i * frame_duration
        elif prob <= threshold and in_utterance:
            in_utterance = False
            utterance_end = i * frame_duration
            if utterance_end - utterance_start > 0:
                utterances.append((utterance_start, utterance_end))

    if in_utterance:
        utterances.append((utterance_start, len(vad_probs) * frame_duration))

    return utterances

def merge_segments(segments, max_duration=8, max_gap=1):
    """Combines segments with pauses shorter than `max_gap` seconds, with total duration limit."""
    merged_segments = []
    if not segments:
        return merged_segments  # Return empty if no segments are found

    current_start, current_end = segments[0]

    for start, end in segments[1:]:
        combined_duration = (end - current_start)

        if (start - current_end <= max_gap) and (combined_duration <= max_duration):
            current_end = end
        else:
            merged_segments.append((current_start, current_end))
            current_start, current_end = start, end

    merged_segments.append((current_start, current_end))
    return merged_segments


def process_audio(audio_file):
    vad_model, _ = torch.hub.load(
        repo_or_dir='snakers4/silero-vad',
        model='silero_vad',
        force_reload=False,
        onnx=False
    )
    vad_model.eval()

    audio, _ = librosa.load(audio_file, sr=sample_rate)
    speech_probs = get_vad_probs(vad_model, audio, sample_rate)
    utterances = get_utterances(speech_probs, threshold=vad_threshold)

    if not utterances:
        print(f"No VAD regions detected for {audio_file}.")
        return
    merged_segments = merge_segments(utterances, max_duration=combine_duration, max_gap=combine_gap)

    if merged_segments:
        return merged_segments
    else:
        return

def transcribe_audio_segment(start_time_sec, end_time_sec):
    # Convert seconds to milliseconds for pydub
    start_time_ms = start_time_sec * 1000
    end_time_ms = end_time_sec * 1000

    # Extract the audio segment
    segment = audio[start_time_ms:end_time_ms]

    # Export segment to an in-memory BytesIO object
    audio_buffer = io.BytesIO()
    segment.export(audio_buffer, format="wav")
    audio_buffer.seek(0)  # Reset buffer position to the beginning

    files = {
        'file': ('audiofile.wav', audio_buffer, 'audio/wav')
    }

    response = requests.post(api_url, headers=headers, files=files, data=data)

    if response.status_code == 200 or response.status_code == 201:
        return response.json()
    else:
        print(f"Error for segment {start_time_sec}-{end_time_sec}: {response.status_code} - {response.text}")
        return None

def write_srt_file(results, output_file_path):
    """
    Writes the transcription results into an SRT file.

    Args:
        results (list): List of dictionaries containing 'start_time', 'end_time', and 'transcript'.
        output_file_path (str): Path to save the SRT file.
    """
    def format_timestamp(seconds):
        """Converts seconds to SRT timestamp format: hh:mm:ss,ms"""
        milliseconds = int((seconds % 1) * 1000)
        seconds = int(seconds)
        minutes = seconds // 60
        hours = minutes // 60
        seconds = seconds % 60
        minutes = minutes % 60
        return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

    with open(output_file_path, "w", encoding="utf-8") as srt_file:
        for i, result in enumerate(results, start=1):
            start_timestamp = format_timestamp(result["start_time"])
            end_timestamp = format_timestamp(result["end_time"])
            transcript = result["transcript"]

            # Write the SRT entry
            srt_file.write(f"{i}\n")
            srt_file.write(f"{start_timestamp} --> {end_timestamp}\n")
            srt_file.write(f"{transcript}\n\n")

api_url = "https://api.sarvam.ai/speech-to-text-translate"
headers = {
    "api-subscription-key" : "d75d7bf3-b053-4084-ac80-c37561a35bfc"
}
data = {
    "model": "saaras:v2",
}
audio_file_path = "test.wav"

timestamps = process_audio(audio_file_path)
audio = AudioSegment.from_file(audio_file_path)

results = []

for start, end in timestamps:
    transcription = transcribe_audio_segment(start, end)
    if transcription is not None:
        results.append({
            "start_time": start,
            "end_time": end,
            "transcript": transcription["transcript"]
        })

# Example usage
output_srt_path = "subtitles.srt"
write_srt_file(results, output_srt_path)

print(f"SRT file has been saved to {output_srt_path}")


Using cache found in /root/.cache/torch/hub/snakers4_silero-vad_master


SRT file has been saved to subtitles.srt
